In [1]:
%cd ..

/home/eli/AnacondaProjects/epych


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych

In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [7]:
def hippocampal_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("DG-" in l or "CA" in l)

def visual_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("VIS" in l)

def subcortical_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("DG-" in l or "CA" in l or "MB" in l or "SCi" in l or "POST" in l)

In [8]:
NWB_FILES = glob.glob('/mnt/data/*.nwb')

In [9]:
PILOT_FILES = []

In [10]:
def samplings():
    cond = CONDITIONS[0]
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if not os.path.exists(basename + "_" + cond):
            continue
        sampling = epych.recording.Sampling.unpickle(basename + "_" + cond).smap(lambda sig: sig.median_filter().select_channels(["VIS" in loc.decode() for loc in sig.channels.location]))
        yield sampling
        del sampling
        logging.info("Loaded LFPs for %s in subject %s" % (cond, nwb_file))

In [11]:
alignment = epych.statistics.alignment.AlignmentSummary()

In [12]:
alignment.calculate(samplings())

INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1169714193_ses-1180116198.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1196154905_ses-1211241460.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1196664789_ses-1212569512.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1196167186_ses-1210866107.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1170220951_ses-1186358749.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1186895119_ses-1204677304.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1197684876_ses-1213341633.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1186895128_ses-1203581890.nwb
INFO:root:Loaded LFPs for go_gloexp in subject /mnt/data/sub-1202441642_ses-1217213788.nwb


{'probeA/VISam': <epych.statistics.alignment.LaminarAlignment at 0x73fa88c79610>,
 'probeB/VISpm': <epych.statistics.alignment.LaminarAlignment at 0x73fc58600950>,
 'probeC/VISp': <epych.statistics.alignment.LaminarAlignment at 0x73fa825f2490>,
 'probeD/VISl': <epych.statistics.alignment.LaminarAlignment at 0x73fa88370e50>,
 'probeE/VISl': <epych.statistics.alignment.LaminarAlignment at 0x73fa829f0110>,
 'probeF/VISrl': <epych.statistics.alignment.LaminarAlignment at 0x73fa88367950>,
 'probeE/VISal': <epych.statistics.alignment.LaminarAlignment at 0x73fa89612510>,
 'probeD/VIS': <epych.statistics.alignment.LaminarAlignment at 0x73fa88691790>,
 'probeE/VISrl': <epych.statistics.alignment.LaminarAlignment at 0x73fa9bffe290>}

In [13]:
alignment.pickle("/mnt/data/mice_passiveglo_alignment")